In [17]:
import json
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer as tw_tokenizer
from unidecode import unidecode
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tag import CRFTagger
from spacy.lang.id import Indonesian
import taudataNlpTm as tau, itertools, re, pickle, pyLDAvis, pyLDAvis.sklearn, spacy, urllib.request
import time, numpy as np, matplotlib.pyplot as plt, networkx as nx, seaborn as sns 
from tqdm import tqdm
from textblob import TextBlob
from pprint import pprint
from datetime import datetime


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases
from gensim.sklearn_api import LdaTransformer
from sklearn.model_selection import GridSearchCV 

#Visualazation
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from datetime import datetime, timedelta

In [18]:
tweets_data = [] 
tweets_file = open('data_tweet_all (batch).txt', "r") 
for line in tweets_file:
    try:
        tweet = json.loads(line)
        if "extended_tweet" in tweet:
            text = tweet["extended_tweet"]["full_text"]
        else:
            text = tweet["text"]
        if "RT @" not in text:
            tweets_data.append(text) 
    except:
        continue
tweets_file.close()

tweets_data[0]

'2021/07/12 20:45:38\n6. PPKM MakeADish\n7. D-573\n8. O.O.O\n9. Official Teaser\n10. Pfizer'

In [19]:
tweets_data = [unidecode(text).lower() for text in tweets_data]
tokenized = [tw_tokenizer().tokenize(text) for text in tweets_data]
data = [[word for word in text if word.isalnum()] for text in tokenized]
print(data[0])

['12', '38', '6', 'ppkm', 'makeadish', '7', 'd', '573', '8', 'o', 'o', 'o', '9', 'official', 'teaser', '10', 'pfizer']


In [20]:
# with open('data/slang.txt', "r", encoding="utf-8", errors='replace') as f:
#     slangwords = f.readlines()

# slangwords = [text.strip('\n').strip().split(':') for word in slangwords]
# slangwords = {key.strip(): value.strip() for key, value in slangwords}

slangwords = dict() 
slangwords_dataframe = pd.read_csv('data/colloquial-indonesian-lexicon.csv') 
for slang, formal in zip(slangwords_dataframe['slang'], slangwords_dataframe['formal']):
    slangwords[slang] = formal 

slangword_file = open('data/slang.txt', "r").read() 
slangwords_text = slangword_file.split('\n')
# print(slangwords_text)
for slang in slangwords_text:
    split_slang = slang.split(":") 
    slangwords[split_slang[0].strip()] = split_slang[1].strip()
# print(slangwords)

for i, text in enumerate(data):    
    for j, word in enumerate(text):
        if word in slangwords.keys():
            data[i][j] = slangwords[word]
#             print(word, slangwords[word], data[i][j])
        
print(data[0])

['12', '38', '6', 'pemberlakuan pembatasan kegiatan masyarakat', 'makeadish', '7', 'di', '573', '8', 'oleh', 'oleh', 'oleh', '9', 'official', 'teaser', '10', 'pfizer']


In [21]:
factory = StopWordRemoverFactory()

Sastrawi_StopWords_id = factory.get_stop_words()

with open('data/stopwords_id.txt', "r", encoding="utf-8", errors='replace') as f:
    stopwords_id = f.readlines()

stopwords_id = [word.strip().lower() for word in stopwords_id]

# remove_stopword = [word for word in data_filtered if word not in Sastrawi_StopWords_id]
# remove_stopword = [word for word in remove_stopword if word not in stopwords_id]
remove_stopword = [[word for word in text if word not in Sastrawi_StopWords_id] for text in data]
remove_stopword = [[word for word in text if word not in stopwords_id] for text in remove_stopword]

print(remove_stopword[0])

['6', 'pemberlakuan pembatasan kegiatan masyarakat', 'makeadish', 'official', 'teaser', 'pfizer']


In [22]:
data_filtered = [" ".join(text) for text in remove_stopword if len(text)>0]
print(data_filtered[0])

6 pemberlakuan pembatasan kegiatan masyarakat makeadish official teaser pfizer


In [23]:
from spacy.lang.id import Indonesian
from nltk.tag import CRFTagger
nlp_id = Indonesian()  # Language Model
ct = CRFTagger()
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')

def NLPfilter(t, filters):
    tokens = nlp_id(t)
    tokens = [str(k) for k in tokens if len(k)>2]
    hasil = ct.tag_sents([tokens])
    return [k[0] for k in hasil[0] if k[1] in filters]

In [24]:
filters = set(['NN', 'NNP', 'NNS', 'NNPS', 'JJ'])

for i, text in enumerate(data_filtered):    
    data_filtered[i] = NLPfilter(text, filters)

data_filtered = [text for text in remove_stopword if len(text)>0]
data_filtered[0]
# for i, d in tqdm(enumerate(data_words)):
#     data_words[i] = NLPfilter(d,filters)

# ' '.join(data_words[0])

['6',
 'pemberlakuan pembatasan kegiatan masyarakat',
 'makeadish',
 'official',
 'teaser',
 'pfizer']

In [25]:
id2words = corpora.Dictionary(data_filtered)
id2words.filter_extremes(no_below = 10, no_above = 0.7)
corpus = [id2words.doc2bow(text) for text in data_filtered]

#print(id2words)
# print(corpus)
# print(id2words.token2id)
print('Number of unique tokens: %d' % len(id2words))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 10010
Number of documents: 135554


In [26]:
print(id2words)
print(corpus[0])

Dictionary(10010 unique tokens: ['6', 'official', 'pemberlakuan pembatasan kegiatan masyarakat', 'pfizer', 'plis']...)
[(0, 1), (1, 1), (2, 1), (3, 1)]


In [27]:
start_time = datetime.now()
olda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                           id2word=id2words,
                           num_topics=3,
                           random_state=100,
                           update_every=1)
print(datetime.now() - start_time)

0:05:17.473317


In [28]:
pprint(olda.print_topics())

[(0,
  '0.041*"covid" + 0.033*"masyarakat" + 0.027*"vaksinasi" + 0.026*"pandemi" + '
  '0.021*"vaksin" + 0.017*"program" + 0.016*"kesehatan" + 0.015*"pemberlakuan '
  'pembatasan kegiatan masyarakat" + 0.013*"ekonomi" + 0.013*"protokol '
  'kesehatan"'),
 (1,
  '0.015*"rakyat" + 0.013*"universitas" + 0.012*"bantuan" + 0.012*"negara" + '
  '0.009*"pembangunan" + 0.009*"data" + 0.008*"lembaga" + 0.008*"papua" + '
  '0.006*"digital" + 0.006*"dewan perwakilan rakyat"'),
 (2,
  '0.094*"vaksin" + 0.042*"merah" + 0.042*"putih" + 0.019*"negeri" + '
  '0.011*"mendukung" + 0.010*"china" + 0.010*"bangsa" + 0.010*"riset" + '
  '0.009*"dukungan" + 0.008*"pabrik"')]


In [29]:
print('\nPerplexity: ', olda.log_perplexity(corpus))


Perplexity:  -7.60338860647679


In [30]:
coherence_model_olda = CoherenceModel(
   model=olda, texts=data_filtered, dictionary=id2words, coherence='c_v'
)
coherence_olda = coherence_model_olda.get_coherence()
print('\nCoherence Score: ', coherence_olda)


Coherence Score:  0.4204019953026756


In [31]:
test_doc = corpus[-1]

vector = olda[test_doc]
print(vector)
#cek lagi

[(0, 0.07201396), (1, 0.8519768), (2, 0.07600924)]


In [32]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(olda, corpus, id2words, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000870  0.298948       1        1  34.962972
1      0.249499 -0.149932       2        1  33.555243
2     -0.248629 -0.149016       3        1  31.481785, topic_info=               Term          Freq         Total Category  logprob  loglift
385          vaksin  51145.000000  51145.000000  Default  30.0000  30.0000
1912          merah  18610.000000  18610.000000  Default  29.0000  29.0000
1279          putih  18483.000000  18483.000000  Default  28.0000  28.0000
44            covid  20002.000000  20002.000000  Default  27.0000  27.0000
351       vaksinasi  13067.000000  13067.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
200       mendukung   4609.598604   7629.389977   Topic3  -4.5466   0.6519
2037           umkm   1625.152256   1814.451175   Topic3  -5.5892   1.0456
3733  mengembangkan   1369.627395   1456.891662   Topic3  -5.7602   1.0940
119          negara   2607.330149   8204.541239   Topic3  -5.1164   0.0094
336           usaha   1559.850622   2862.110416   Topic3  -5.6302   0.5488

[213 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
9769      1  0.001203            395
9769      2  0.998414            395
104       2  0.998277          agama
104       3  0.001272          agama
2334      1  0.162705      airlangga
...     ...       ...            ...
5841      3  0.995878  wajib militer
138       1  0.319729          warga
138       2  0.580353          warga
138       3  0.099974          warga
4771      1  0.999253             xx

[368 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])